# import / define

In [13]:
import numpy as np
from sklearn.cluster import KMeans
import sys
import glob
import cv2
import igraph as ig
import os
from matplotlib import pyplot as plt

In [14]:
class GaussianMixture:
    def __init__(self, X, n_components):
        self.n_components = n_components
        self.n_features = X.shape[1]
        self.n_samples = np.zeros(self.n_components)

        self.coefs = np.zeros(self.n_components)
        self.means = np.zeros((self.n_components, self.n_features))
       
        self.covariances = np.zeros((self.n_components, self.n_features, self.n_features))
        self.init_with_kmeans(X)

    def init_with_kmeans(self, X):
        label = KMeans(n_clusters=self.n_components, n_init=1).fit(X).labels_
        self.fit(X, label)

    def calc_score(self, X, ci):
        score = np.zeros(X.shape[0])
        
        if self.coefs[ci] > 0:
            diff = X - self.means[ci]
            mult = np.einsum('ij,ij->i', diff, np.dot(np.linalg.inv(self.covariances[ci]), diff.T).T)
            score = np.exp(-.5 * mult) / np.sqrt(2 * np.pi) / np.sqrt(np.linalg.det(self.covariances[ci]))
        
        return score

    def calc_prob(self, X):
        prob = [self.calc_score(X, ci) for ci in range(self.n_components)]
        
        return np.dot(self.coefs, prob)

    def which_component(self, X):
        prob = np.array([self.calc_score(X, ci) for ci in range(self.n_components)]).T

        return np.argmax(prob, axis=1)

    def fit(self, X, labels):
        assert self.n_features == X.shape[1]

        self.n_samples[:] = 0
        self.coefs[:] = 0

        uni_labels, count = np.unique(labels, return_counts=True)
        self.n_samples[uni_labels] = count

        variance = 0.01
        for ci in uni_labels:
            n = self.n_samples[ci]

            self.coefs[ci] = n / np.sum(self.n_samples)
            self.means[ci] = np.mean(X[ci == labels], axis=0)
            self.covariances[ci] = 0 if self.n_samples[ci] <= 1 else np.cov(X[ci == labels].T)

            det = np.linalg.det(self.covariances[ci])
            if det <= 0:
                self.covariances[ci] += np.eye(self.n_features) * variance
                det = np.linalg.det(self.covariances[ci])

In [15]:
class GrabCut:
    def __init__(self, img, mask,num_iters, gamma,gmm_components,flag,rect=None):
        self.img = np.asarray(img, dtype=np.float64)
        self.rows, self.cols, _ = img.shape
        self.mask = mask
        
        if rect is not None:
            self.mask[rect[1]:rect[1] + rect[3],rect[0]:rect[0] + rect[2]] = DRAW_PR_FG['val']
        
        self.classify_pixels()

        # Best number of GMM components K suggested in paper
        self.gmm_components = gmm_components
        
        # Best gamma suggested in paper formula (5)
        self.gamma = gamma 
        
        self.beta = 0

        self.left_V = np.empty((self.rows, self.cols - 1))
        self.upright_V = np.empty((self.rows - 1, self.cols - 1))
        
        if flag==8:
            self.upleft_V = np.empty((self.rows - 1, self.cols - 1))
            self.up_V = np.empty((self.rows - 1, self.cols))

        self.bgd_gmm = None
        self.fgd_gmm = None
        self.comp_idxs = np.empty((self.rows, self.cols), dtype=np.uint32)

        self.gc_graph = None
        self.gc_graph_capacity = None           # Edge capacities
        self.gc_source = self.cols * self.rows  # "object" terminal S
        self.gc_sink = self.gc_source + 1       # "background" terminal T

        self.calc_beta_smoothness()
        self.init_GMMs()
        self.run(num_iters)

    def calc_beta_smoothness(self):
        _left_diff = self.img[:, 1:] - self.img[:, :-1]
        _up_diff = self.img[1:, :] - self.img[:-1, :]
        
        if flag==8:
            _upleft_diff = self.img[1:, 1:] - self.img[:-1, :-1]
            _upright_diff = self.img[1:, :-1] - self.img[:-1, 1:]   
            self.beta = np.sum(np.square(_left_diff)) + np.sum(np.square(_upleft_diff)) + \
                np.sum(np.square(_up_diff)) + np.sum(np.square(_upright_diff))
            self.beta = 1 / (2 * self.beta / (
                # Each pixel has 4 neighbors (left, upleft, up, upright)
                4 * self.cols * self.rows
                # The 1st column doesn't have left, upleft and the last column doesn't have upright
                - 3 * self.cols
                - 3 * self.rows  # The first row doesn't have upleft, up and upright
                + 2))  # The first and last pixels in the 1st row are removed twice
        else:
            self.beta = np.sum(np.square(_left_diff)) + np.sum(np.square(_up_diff)) 
            self.beta = 1 / (2 * self.beta / (
            # Each pixel has 2 neighbors (left,up)
            2 * self.cols * self.rows
            # The 1st column doesn't have left
            - self.cols
            # The first row doesn't have up
            - self.rows))
#         print('Beta:', self.beta)

        # Smoothness term V described in formula (11)
        self.left_V = self.gamma * np.exp(-self.beta * np.sum(np.square(_left_diff), axis=2))
        self.up_V = self.gamma * np.exp(-self.beta * np.sum(np.square(_up_diff), axis=2))
        
        if flag==8:
            self.upleft_V = self.gamma / np.sqrt(2) * np.exp(-self.beta * np.sum(np.square(_upleft_diff), axis=2))
            self.upright_V = self.gamma / np.sqrt(2) * np.exp(-self.beta * np.sum(np.square(_upright_diff), axis=2))


    def classify_pixels(self):
        self.bgd_indexes = np.where(np.logical_or(self.mask == DRAW_BG['val'], self.mask == DRAW_PR_BG['val']))
        self.fgd_indexes = np.where(np.logical_or(self.mask == DRAW_FG['val'], self.mask == DRAW_PR_FG['val']))

        assert self.bgd_indexes[0].size > 0
        assert self.fgd_indexes[0].size > 0

#         print('(pr_)bgd count: %d, (pr_)fgd count: %d' % (self.bgd_indexes[0].size, self.fgd_indexes[0].size))

    def init_GMMs(self):
        self.bgd_gmm = GaussianMixture(self.img[self.bgd_indexes],gmm_components)
        self.fgd_gmm = GaussianMixture(self.img[self.fgd_indexes],gmm_components)

    def assign_GMMs_components(self):
        #Step 1 in Figure 3: Assign GMM components to pixels
        self.comp_idxs[self.bgd_indexes] = self.bgd_gmm.which_component(self.img[self.bgd_indexes])
        self.comp_idxs[self.fgd_indexes] = self.fgd_gmm.which_component(self.img[self.fgd_indexes])

    def learn_GMMs(self):
        #Step 2 in Figure 3: Learn GMM parameters from data z
        self.bgd_gmm.fit(self.img[self.bgd_indexes],self.comp_idxs[self.bgd_indexes])
        self.fgd_gmm.fit(self.img[self.fgd_indexes],self.comp_idxs[self.fgd_indexes])

    def construct_gc_graph(self):
        bgd_indexes = np.where(self.mask.reshape(-1) == DRAW_BG['val'])
        fgd_indexes = np.where(self.mask.reshape(-1) == DRAW_FG['val'])
        pr_indexes = np.where(np.logical_or(self.mask.reshape(-1) == DRAW_PR_BG['val'], self.mask.reshape(-1) == DRAW_PR_FG['val']))

#         print('bgd count: %d, fgd count: %d, uncertain count: %d' % (len(bgd_indexes[0]), len(fgd_indexes[0]), len(pr_indexes[0])))

        edges = []
        self.gc_graph_capacity = []

        # t-links
        edges.extend(
            list(zip([self.gc_source] * pr_indexes[0].size, pr_indexes[0])))
        _D = -np.log(self.bgd_gmm.calc_prob(self.img.reshape(-1, 3)[pr_indexes]))
        self.gc_graph_capacity.extend(_D.tolist())
        assert len(edges) == len(self.gc_graph_capacity)

        edges.extend(
            list(zip([self.gc_sink] * pr_indexes[0].size, pr_indexes[0])))
        _D = -np.log(self.fgd_gmm.calc_prob(self.img.reshape(-1, 3)[pr_indexes]))
        self.gc_graph_capacity.extend(_D.tolist())
        assert len(edges) == len(self.gc_graph_capacity)

        edges.extend(
            list(zip([self.gc_source] * bgd_indexes[0].size, bgd_indexes[0])))
        self.gc_graph_capacity.extend([0] * bgd_indexes[0].size)
        assert len(edges) == len(self.gc_graph_capacity)

        edges.extend(
            list(zip([self.gc_sink] * bgd_indexes[0].size, bgd_indexes[0])))
        self.gc_graph_capacity.extend([9 * self.gamma] * bgd_indexes[0].size)
        assert len(edges) == len(self.gc_graph_capacity)

        edges.extend(
            list(zip([self.gc_source] * fgd_indexes[0].size, fgd_indexes[0])))
        self.gc_graph_capacity.extend([9 * self.gamma] * fgd_indexes[0].size)
        assert len(edges) == len(self.gc_graph_capacity)

        edges.extend(
            list(zip([self.gc_sink] * fgd_indexes[0].size, fgd_indexes[0])))
        self.gc_graph_capacity.extend([0] * fgd_indexes[0].size)
        assert len(edges) == len(self.gc_graph_capacity)

        # n-links
        img_indexes = np.arange(self.rows * self.cols,dtype=np.uint32).reshape(self.rows, self.cols)

        mask1 = img_indexes[:, 1:].reshape(-1)
        mask2 = img_indexes[:, :-1].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(self.left_V.reshape(-1).tolist())
        assert len(edges) == len(self.gc_graph_capacity)

        mask1 = img_indexes[1:, :].reshape(-1)
        mask2 = img_indexes[:-1, :].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(self.up_V.reshape(-1).tolist())
        assert len(edges) == len(self.gc_graph_capacity)
        
        if flag==8:
        
            mask1 = img_indexes[1:, 1:].reshape(-1)
            mask2 = img_indexes[:-1, :-1].reshape(-1)
            edges.extend(list(zip(mask1, mask2)))
            self.gc_graph_capacity.extend(self.upleft_V.reshape(-1).tolist())
            assert len(edges) == len(self.gc_graph_capacity)

            mask1 = img_indexes[1:, :-1].reshape(-1)
            mask2 = img_indexes[:-1, 1:].reshape(-1)
            edges.extend(list(zip(mask1, mask2)))
            self.gc_graph_capacity.extend(self.upright_V.reshape(-1).tolist())
            assert len(edges) == len(self.gc_graph_capacity)
            
            assert len(edges) == 4 * self.cols * self.rows - 3 * (self.cols + self.rows) + 2 +2 * self.cols * self.rows
            
            
        else:
            assert len(edges) == 2 * self.cols * self.rows - (self.cols + self.rows) + 2* self.cols * self.rows

        
        self.gc_graph = ig.Graph(self.cols * self.rows + 2)
        self.gc_graph.add_edges(edges)

    def estimate_segmentation(self):
        #Step 3 in Figure 3: Estimate segmentation
        mincut = self.gc_graph.st_mincut(self.gc_source, self.gc_sink, self.gc_graph_capacity)
#         print('foreground pixels: %d, background pixels: %d' % (len(mincut.partition[0]), len(mincut.partition[1])))
        pr_indexes = np.where(np.logical_or(self.mask == DRAW_PR_BG['val'], self.mask == DRAW_PR_FG['val']))
        img_indexes = np.arange(self.rows * self.cols, dtype=np.uint32).reshape(self.rows, self.cols)
        self.mask[pr_indexes] = np.where(np.isin(img_indexes[pr_indexes], mincut.partition[0]),DRAW_PR_FG['val'], DRAW_PR_BG['val'])
        self.classify_pixels()
        
    def run(self, num_iters):
        for _ in range(num_iters):
            self.assign_GMMs_components()
            self.learn_GMMs()
            self.construct_gc_graph()
            self.estimate_segmentation()

In [16]:
# Defining variables and flags
# BLUE = [255, 0, 0]
DRAW_BG = { 'val': 0}
DRAW_FG = { 'val': 1}
DRAW_PR_FG = {'val': 3}
DRAW_PR_BG = {'val': 2}

# setting up flags
rect = (0, 0, 1, 1)
rectangle = False       # flag for drawing rect
rect_over = False       # flag to check if rect drawn

In [17]:
filename="./masked_dataset_face_cropped/16/n_16_fron_face.jpg"

gmm_components=5
num_iters=1
gamma=50
flag=8
img = cv2.imread(filename)

# auto_grabcut
img2 = img.copy() # a copy of original image

mask = np.zeros(img.shape[:2], dtype=np.uint8)  # mask initialized to BG
auto_grabcut = np.zeros(img.shape, np.uint8)    # output image to be shown

print(img.shape)

rect = (1, 1, img.shape[1]-2, img.shape[0]-2)
rectangle = True # test
rect_over =  True # test
print(rect, rect)
gc = GrabCut(img2, mask,num_iters,gamma,gmm_components, flag,rect)

mask2 = np.where((mask == 1) + (mask == 3), 255, 0).astype('uint8')
auto_grabcut = cv2.bitwise_and(img2, img2, mask=mask2)

# hsv_mask
hsv = cv2.cvtColor(auto_grabcut, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)


s = cv2.inRange(s, 0, 50) ##### hsv 범위 조절
hsv_mask = cv2.bitwise_and(hsv, hsv, mask = s)
hsv_mask = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

# calculate ratio
img_gray = cv2.cvtColor(hsv_mask, cv2.COLOR_BGR2GRAY)
img_bin = np.zeros(img_gray.shape)

no_mask_px_cnt = 0
mask_px_cnt = 0

for i in range(0, img_gray.shape[0]):
    for j in range(0, img_gray.shape[1]):
        if img_gray[i][j] == 0:
            img_bin[i][j] = 0 # no_mask_px
            no_mask_px_cnt = no_mask_px_cnt + 1
        else:
            img_bin[i][j] = 255 # mask_px
            mask_px_cnt = mask_px_cnt + 1

print("the ratio of mask:", mask_px_cnt/(mask_px_cnt+no_mask_px_cnt)*100, "%")
# 마스크로 가려진 부분의 비율(단위: %)


# 결과 출력
cv2.namedWindow("1. input", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("1. input", img)

cv2.namedWindow("2. auto_grabcut", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("2. auto_grabcut", auto_grabcut)

cv2.namedWindow("3. hsv", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("3. hsv", hsv_mask)

cv2.namedWindow("4. binary image", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("4. binary image",img_bin)

cv2.waitKey(0)
cv2.destroyAllWindows()

(120, 120, 3)
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 27.0625 %


# 반복문으로 여러 이미지의 평균 비율 구하기

In [30]:
import os

# 마스크 검출이 잘 되는 사람 폴더
folder_list = ['03', '04', '05', '06', '08', '09', '10', '14', '15', '17', '20', '21', '24', '25', '28', '31', '32', '34', '36', '38']

sum_ratio = 0
# 마스크 비율을 계산할 이미지들
for i in folder_list:
    path_dir = "./masked_dataset_face_cropped/" + i
    file_list = os.listdir(path_dir)

    for j in range(len(file_list)):
        filename = file_list[j]
        file_path = path_dir + "/" + filename
        
        gmm_components=5
        num_iters=1
        gamma=50
        flag=8
    #     img = cv2.imread(filename)
        img = cv2.imread(file_path)
        print(file_path)

        # auto_grabcut
        img2 = img.copy() # a copy of original image
        mask = np.zeros(img.shape[:2], dtype=np.uint8)  # mask initialized to BG
        auto_grabcut = np.zeros(img.shape, np.uint8)    # output image to be shown
    #     print(img.shape)

        rect = (1, 1, img.shape[1]-2, img.shape[0]-2)
        rectangle = True # test
        rect_over =  True # test
        print(rect, rect)
        gc = GrabCut(img2, mask,num_iters,gamma,gmm_components, flag,rect)

        mask2 = np.where((mask == 1) + (mask == 3), 255, 0).astype('uint8')
        auto_grabcut = cv2.bitwise_and(img2, img2, mask=mask2)

        # hsv_mask
        hsv = cv2.cvtColor(auto_grabcut, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        s = cv2.inRange(s, 0, 50) ##### hsv 범위 조절
        hsv_mask = cv2.bitwise_and(hsv, hsv, mask = s)
        hsv_mask = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

        # calculate ratio
        img_gray = cv2.cvtColor(hsv_mask, cv2.COLOR_BGR2GRAY)
        img_bin = np.zeros(img_gray.shape)

        no_mask_px_cnt = 0
        mask_px_cnt = 0

        for i in range(0, img_gray.shape[0]):
            for j in range(0, img_gray.shape[1]):
                if img_gray[i][j] == 0:
                    img_bin[i][j] = 0 # no_mask_px
                    no_mask_px_cnt = no_mask_px_cnt + 1
                else:
                    img_bin[i][j] = 255 # mask_px
                    mask_px_cnt = mask_px_cnt + 1
                    ratio = mask_px_cnt/(mask_px_cnt+no_mask_px_cnt)
        sum_ratio = sum_ratio + ratio

        print("the ratio of mask:", ratio*100, "%")
        print("")

avg_ratio = sum_ratio / (len(folder_list) * len(file_list))        
print("전체 비율: ", avg_ratio*100, '%')
# 마스크로 가려진 부분의 비율(단위: %)


# 결과 출력
# cv2.namedWindow("1. input", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("1. input", img)

# cv2.namedWindow("2. auto_grabcut", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("2. auto_grabcut", auto_grabcut)

# cv2.namedWindow("3. hsv", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("3. hsv", hsv_mask)

# cv2.namedWindow("4. binary image", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("4. binary image",img_bin)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

./masked_dataset_face_cropped/03/f_03_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 37.94245171462357 %

./masked_dataset_face_cropped/03/f_03_fron_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 27.184185845177115 %

./masked_dataset_face_cropped/03/f_03_topp_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 20.854071990189713 %

./masked_dataset_face_cropped/03/n_03_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 23.990642215681834 %

./masked_dataset_face_cropped/03/n_03_fron_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 24.31469497564168 %

./masked_dataset_face_cropped/03/n_03_topp_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 18.523019731198172 %

./masked_dataset_face_cropped/04/f_04_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 39.276563677467976 %

./masked_dataset_face_cropped/04/f_04_fron_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 28.93509

C:\Users\KIMA_GRAM\Anaconda3\envs\ny_tf\lib\site-packages\ipykernel_launcher.py:112: RuntimeWarning: divide by zero encountered in log


the ratio of mask: 29.659769045958306 %

./masked_dataset_face_cropped/17/n_17_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 26.05062082139446 %

./masked_dataset_face_cropped/17/n_17_fron_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 26.822289156626507 %

./masked_dataset_face_cropped/17/n_17_topp_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 24.657635650307203 %

./masked_dataset_face_cropped/20/f_20_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 41.738995660260386 %

./masked_dataset_face_cropped/20/f_20_fron_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 20.867163304595294 %

./masked_dataset_face_cropped/20/f_20_topp_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 18.661137440758292 %

./masked_dataset_face_cropped/20/h_20_bott_face.jpg
(1, 1, 118, 118) (1, 1, 118, 118)
the ratio of mask: 37.644285482035436 %

./masked_dataset_face_cropped/20/h_20_fron_face.jpg
(1, 1, 118, 118) (1

# 여기까지 실행 (Cell - Run All Above)

---

# 1단계. 배경영역 제거하기 auto_grabcut

In [8]:
filename="./masked_dataset_face_cropped/35/f_35_fron_face.jpg"
gmm_components=5
num_iters=1
gamma=50
flag=8
img = cv2.imread(filename)

img2 = img.copy() # a copy of original image

mask = np.zeros(img.shape[:2], dtype=np.uint8)  # mask initialized to BG
auto_grabcut = np.zeros(img.shape, np.uint8)           # output image to be shown

# # input and output windows
# cv2.namedWindow('output')
# cv2.namedWindow('input')
# # cv2.setMouseCallback('input', onmouse)
# cv2.moveWindow('input', img.shape[1]+10, 90)

print(img.shape)

# print(" Instructions: \n")
# print(" Draw a rectangle around the object using right mouse button \n")

rect = (1, 1, img.shape[1]-2, img.shape[0]-2)
rectangle = True # test
rect_over =  True # test
print(rect, rect)
gc = GrabCut(img2, mask,num_iters,gamma,gmm_components, flag,rect)

mask2 = np.where((mask == 1) + (mask == 3), 255, 0).astype('uint8')
auto_grabcut = cv2.bitwise_and(img2, img2, mask=mask2)

# 결과 출력
# cv2.namedWindow("input", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("input", img)

# cv2.namedWindow("auto_grabcut", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
# cv2.imshow("auto_grabcut", auto_grabcut)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

(120, 120, 3)
(1, 1, 118, 118) (1, 1, 118, 118)
(pr_)bgd count: 476, (pr_)fgd count: 13924
Beta: 0.0004286698417465072
bgd count: 476, fgd count: 0, uncertain count: 13924
foreground pixels: 9241, background pixels: 5161
(pr_)bgd count: 5160, (pr_)fgd count: 9240


# 2단계. 마스크 영역만 검출하기(HSV)

In [11]:
hsv = cv2.cvtColor(auto_grabcut, cv2.COLOR_BGR2HSV)

h, s, v = cv2.split(hsv)
print(s)

s = cv2.inRange(s, 0, 30)
hsv_mask = cv2.bitwise_and(hsv, hsv, mask = s)
hsv_mask = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

# 결과 출력
cv2.namedWindow("auto_grabcut", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("auto_grabcut", auto_grabcut)

cv2.namedWindow("hsv", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("hsv", hsv_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# 3단계. 마스크 영역 비율 계산

In [18]:
img_gray = cv2.cvtColor(hsv_mask, cv2.COLOR_BGR2GRAY)
img_bin = np.zeros(img_gray.shape)

no_mask_px_cnt = 0
mask_px_cnt = 0

for i in range(0, img_gray.shape[0]):
    for j in range(0, img_gray.shape[1]):
        if img_gray[i][j] == 0:
            img_bin[i][j] = 0 # no_mask_px
            no_mask_px_cnt = no_mask_px_cnt + 1
        else:
            img_bin[i][j] = 255 # mask_px
            mask_px_cnt = mask_px_cnt + 1

print("the ratio of mask:", mask_px_cnt/(mask_px_cnt+no_mask_px_cnt)*100, "%")
# 마스크로 가려진 부분의 비율(단위: %)

# 결과 출력
cv2.namedWindow("gray image", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("gray image",img_gray)

cv2.namedWindow("binary image", cv2.WINDOW_NORMAL) # 창 크기 조절 가능
cv2.imshow("binary image",img_bin)

cv2.waitKey(0)
cv2.destroyAllWindows()

the ratio of mask: 29.75 %


# 피부 영역 검출

In [7]:
# import cv2
# import numpy as np

# #Open a simple image
# img=cv2.imread("f_19_bott_periocular.jpg")

# #converting from gbr to hsv color space
# img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# #skin color range for hsv color space 
# HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 
# HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

# #converting from gbr to YCbCr color space
# img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
# #skin color range for hsv color space 
# YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
# YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

# #merge skin detection (YCbCr and hsv)
# global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
# global_mask=cv2.medianBlur(global_mask,3)
# global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))


# HSV_result = cv2.bitwise_not(HSV_mask)
# YCrCb_result = cv2.bitwise_not(YCrCb_mask)
# global_result=cv2.bitwise_not(global_mask)


# #show results
# cv2.imshow("1_HSV.jpg",HSV_result)
# cv2.imshow("2_YCbCr.jpg",YCrCb_result)
# cv2.imshow("3_global_result.jpg",global_result)
# cv2.imshow("Image.jpg",img)
# # cv2.imwrite("1_HSV.jpg",HSV_result)
# # cv2.imwrite("2_YCbCr.jpg",YCrCb_result)
# # cv2.imwrite("3_global_result.jpg",global_result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()  